# Plot main project figures 

In [1]:
library_load <- suppressMessages(
    
    list(
        
        # Seurat 
        library(Seurat), 
        
        # Data
        library(tidyverse), 
        
        # Plotting 
        library(ggplot2), 
        library(patchwork)
        
    )
)

In [2]:
random_seed <- 42
set.seed(random_seed)

In [3]:
options(warn=-1)

In [4]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [5]:
# Source files
source("plotting_global.R")

## Parameter settings

In [6]:
# Files and objects 
so_int_file <- "data/object/seurat_int.rds"

# Plotting Theme
ggplot2::theme_set(theme_global_set()) # From project global source()

## Import Seurat object

In [7]:
so_int <- readRDS(so_int_file)

# Integrated analysis 

### Jak-Stat expression 

In [8]:
rownames(so_int)[grep("Ifit", rownames(so_int))]
rownames(so_int)[grep("Mx1", rownames(so_int))]
rownames(so_int)[grep("Eif", rownames(so_int))]

[1] "Ifitm1"   "Ifitm3"   "Ifitm6"   "Ifitm2"   "Ifit1"    "Ifit3"    "Ifit3b"  
[8] "Ifit1bl1" "Ifit2"

[1] "Mx1"

[1] "Eif4g3" "Eif4a2" "Eif4e3" "Eif5a"

In [ ]:
features_ifn <- rownames(so_int)[grep("Ifn", rownames(so_int))]
features_stat <- rownames(so_int)[grep("Stat", rownames(so_int))]
features_irf <- rownames(so_int)[grep("Irf", rownames(so_int))]
features_ifit <- rownames(so_int)[grep("Ifit", rownames(so_int))]
features_mx1 <- rownames(so_int)[grep("Mx1", rownames(so_int))]
features_eif <- rownames(so_int)[grep("Eif", rownames(so_int))]

features <- c(features_ifn, features_stat, features_irf, features_ifit, features_mx1, features_eif)
arrange <- c(
    
    'Ifngr1','Ifngr2','Ifnar1','Ifnar2',
    'Stat1','Stat2','Stat3','Stat4','Stat6',
    'Irf1','Irf2','Irf2bp2','Irf2bpl','Irf4','Irf7','Irf5','Irf8',
    'Ifit1','Ifitm1','Ifit1bl1','Ifit2','Ifitm2','Ifit3','Ifitm3','Ifit3b','Ifitm6',
    'Eif4a2','Eif4g3','Eif4e3','Eif5a'
    
)
features <- features[arrange]

cnt <- GetAssayData(subset(so_int, features=features), assay="RNA", slot="data") %>% as.data.frame() %>% mutate(gene=rownames(.))
cnt <- pivot_longer(cnt, !gene, names_to="cell_id", values_to="counts")
meta <- so_int@meta.data %>% mutate(cell_id=rownames(.))
data <- dplyr::left_join(cnt, meta, by="cell_id")

In [ ]:
options(repr.plot.width=15, repr.plot.height=17)
ggplot(data[data$counts>0, ], aes(x=integrated_snn_res.0.8, y=counts, color=treatment)) + 
    geom_jitter(color="gray", size=0.1) +
    geom_boxplot() + 
    xlab("cluster ID") + ylab("normalized counts") +
    scale_color_manual(values=color$treatment) + 
    facet_wrap(~gene, ncol=2)

In [ ]:
fp_nacl <- list()
fp_cpg <- list()
for(f in features) {
    
    so_split <- SplitObject(so_int, split.by="treatment")
    
    fp_nacl[[f]] <- FeaturePlot(so_split[["NaCl"]], reduction="umap", features=f, order=TRUE, cols=c("lightgrey", color$treatment["NaCl"])) & theme(aspect.ratio=1)
    fp_cpg[[f]] <- FeaturePlot(so_split[["CpG"]], reduction="umap", features=f, order=TRUE, cols=c("lightgrey", color$treatment["CpG"])) & theme(aspect.ratio=1) 
    
}

In [ ]:
options(repr.plot.width=25, repr.plot.height=7.5)
wrap_plots(fp_nacl, ncol=10)
wrap_plots(fp_cpg, ncol=10)

# Session info

In [ ]:
sessionInfo()